In [8]:
import gtfs_kit as gk

# 载入GTFS数据
feed = gk.read_feed('gtfs.zip', dist_units='m')


In [12]:
import geopandas as gpd
from shapely.geometry import Point

# 假设A点坐标
point_a = Point(144.962646, -37.810272)

# 创建地理数据框
stations = gpd.GeoDataFrame(feed.stops, geometry=gpd.points_from_xy(feed.stops.stop_lon, feed.stops.stop_lat))
stations['distance'] = stations.geometry.distance(point_a)
nearest_station = stations.loc[stations['distance'].idxmin()]


In [13]:
nearest_station

stop_id                                            18031
stop_name    5-Elizabeth St/La Trobe St (Melbourne City)
stop_lat                                      -37.810291
stop_lon                                      144.961596
geometry      POINT (144.961595632073 -37.8102909785482)
distance                                        0.001051
Name: 213, dtype: object